In [ ]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [ ]:
''' MOUNTING '''
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_doc(filename):
    # open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [ ]:
# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

In [ ]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

In [ ]:
def preprocess(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = image.load_img(image_path, target_size=(299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [ ]:
# Function to encode a given image into a vector of size (2048, )
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

In [ ]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

In [ ]:
# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

In [ ]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        print("while loops")
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            print("for 1")

            # print("nama photo :" , photos)
            # print(type(photos))
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                print("for 2")
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    print("for 3")
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)

            # yield the batch data
            if n==num_photos_per_batch:
                print("here i")
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

In [ ]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [ ]:
# load training dataset (6K)
filename = '/content/drive/My Drive/Flickr8k/Flickr_500/list_train.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))

Dataset: 495


In [ ]:
# Below path contains all the images
images = '/content/drive/My Drive/Flickr8k/Flickr_500/'
# Create a list of all image names in the directory
img = glob.glob(images + '*.jpg')

In [ ]:
# Below file conatains the names of images to be used in train data
train_images_file = '/content/drive/My Drive/Flickr8k/Flickr_500/list_train.txt'
# Read the train image names in a set
train_images = set(open(train_images_file, 'r').read().strip().split('\n'))

# Create a list of all the training images with their full path names
train_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in train_images: # Check if the image belongs to training set
        train_img.append(i) # Add it to the list of train images

In [ ]:
# Below file conatains the names of images to be used in test data
test_images_file = '/content/drive/My Drive/Flickr8k/Flickr_500/test/list_test.txt'
# Read the validation image names in a set# Read the test image names in a set
test_images = set(open(test_images_file, 'r').read().strip().split('\n'))

# Create a list of all the test images with their full path names
test_img = []

for i in img: # img is list of full path names of all images
  # print(i)
    if i[len(images):] in test_images: # Check if the image belongs to test set
        test_img.append(i) # Add it to the list of test images

In [ ]:
# descriptions
file_desc = '/content/drive/My Drive/Flickr8k/Flickr_500/description/descriptions_by_vita_windi_olah_1_tab.csv'
train_descriptions = load_clean_descriptions(file_desc, train)
print('Descriptions: train=%d' % len(train_descriptions))

# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

Descriptions: train=316
Description Length: 24


In [ ]:
# Load the inception v3 model
model = InceptionV3(weights='imagenet')

# Create a new model, by removing the last layer (output layer) from the inception v3
model_new = Model(model.input, model.layers[-2].output)

In [ ]:
''' JALANKAN JIKA BELUM PUNYA PKL SAJA  ==> SUDAH '''
# Call the funtion to encode all the train images
# This will take a while on CPU - Execute this only once
start = time()
encoding_train = {}
for img in train_img:
    encoding_train[img[len(images):]] = encode(img)
print("Time taken in seconds =", time()-start)

# Save the bottleneck train features to disk
file_train_pkl = '/content/drive/My Drive/Flickr8k/Flickr_500/encoded_train_images.pkl'
with open(file_train_pkl, "wb") as encoded_pickle:
    dump(encoding_train, encoded_pickle)

In [ ]:
''' JALANKAN JIKA BELUM PUNYA PKL SAJA => SUDAH '''
# Call the funtion to encode all the test images - Execute this only once
start = time()
encoding_test = {}
for img in test_img:
    encoding_test[img[len(images):]] = encode(img)
print("Time taken in seconds =", time()-start)

# Save the bottleneck test features to disk
file_test_pkl = '/content/drive/My Drive/Flickr8k/Flickr_500/encoded_test_images.pkl'
with open(file_test_pkl, "wb") as encoded_pickle:
    dump(encoding_test, encoded_pickle)

In [ ]:
file_train_pkl = '/content/drive/My Drive/Flickr8k/Flickr_500/encoded_train_images.pkl'
train_features = load(open(file_train_pkl, "rb"))
print('Photos: train=%d' % len(train_features))  

In [ ]:
# Create a list of all the training captions
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
len(all_train_captions)

In [ ]:
# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

In [ ]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1
    
vocab_size = len(ixtoword) + 1 # one for appended 0's

In [ ]:
# Load Fast Text vectors
wordembedding_dir = "/content/drive/My Drive/Flickr8k/"
embeddings_index = {} # empty dictionary
f = open(os.path.join(wordembedding_dir, 'cc.id.300.vec'), encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
embedding_dim = 300

# Get 300-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector
        
# embedding_matrix.shape

In [ ]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

# model.summary()
# model.layers[2]

model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# gen = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
# print(type(gen))

In [ ]:
model.optimizer.lr.value = 0.0001
epochs = 10
number_pics_per_bath = 3 # 6
steps = len(train_descriptions) #number_pics_per_bath

file_model = '/content/drive/My Drive/Flickr8k/Flickr_500/model/'

print("train descriptions :", type(train_descriptions))
print("wordtoix : ", type(wordtoix))
print("max length : ", type(max_length))
print("number pics per bath : ", type(number_pics_per_bath))
print("train features : ", type(train_features))

for i in range(epochs):
    print("i : ", i)
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    # generator
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    # model.save(file_model + 'model_' + str(i) + '.h5')

file_last_model = file_model + 'model_' + str(epochs) + '.h5'
model.save_weights(file_last_model)

In [ ]:
file_test_pkl = '/content/drive/My Drive/Flickr8k/Flickr_500/encoded_test_images.pkl'
with open(file_test_pkl, "rb") as encoded_pickle:
    encoding_test = load(encoded_pickle)
    
z +=1
pic = list(encoding_test.keys())[z]
image = encoding_test[pic].reshape((1,2048))
x=plt.imread(images+pic)
plt.imshow(x)
plt.show()
print("Greedy:",greedySearch(image))